In [ ]:
from typing import List
import sys
import numpy as np
from IPython import display
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import os


from main import dataloader, to_vect, Coordinates, dataloader_2024

Loading data from : C:\Users\HP\Desktop\Cours\Projet IA\data_2022\trajectories_processed
Loading data from : C:\Users\HP\Desktop\Cours\Projet IA\data_2024\trajectories_processed


In [3]:
def transform(coords: List[Coordinates], offset: np.ndarray, factor: float) -> np.ndarray:
    return (to_vect(coords) + offset) * factor

def draw_cross(draw: ImageDraw.Draw, coord: tuple[int, int], time: int):
    offset = 10
    topleft = (coord[0] - offset, coord[1] - offset)
    topright = (coord[0] + offset, coord[1] - offset)
    bottomleft = (coord[0] - offset, coord[1] + offset)
    bottomright = (coord[0] + offset, coord[1] + offset)
    draw.line(topleft + bottomright, fill=(0, 0, 255), width=4)
    draw.line(topright + bottomleft, fill=(0, 0, 255), width=4)
    font = ImageFont.load_default()
    draw.text((coord[0] + 30, coord[1]), anchor="mm", fill="blue",
              text=f"{time}mn", font=font, align="center")

In [4]:
print(next(iter(dataloader)).id)

Giorgia


In [ ]:
##Données 2022

from xml.sax.handler import all_features


notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir) 
parent_parent_dir = os.path.dirname(parent_dir)      

images_dir = os.path.join(parent_parent_dir, "images_2022")  
os.makedirs(images_dir, exist_ok=True)
       
no_mushroom_dir = os.path.join(images_dir, "no_mushroom")
no_timestamp_dir = os.path.join(images_dir, "no_timestamp")
no_mushroom_no_timestamp_dir = os.path.join(images_dir, "no_mushroom_no_timestamp")
all_features_dir = os.path.join(images_dir, "all_features")

os.makedirs(no_mushroom_dir, exist_ok=True)
os.makedirs(no_timestamp_dir, exist_ok=True)
os.makedirs(no_mushroom_no_timestamp_dir, exist_ok=True)
os.makedirs(all_features_dir, exist_ok=True)


for idx, sample in enumerate(dataloader):
    session = sample["patchy"]
    if session is None:
        print("No session")
        continue

    ZOOM_FACTOR = 1.5

    # Transformation des coordonnées
    mushrooms_coords = transform(session.mushroom_coords, np.array([200, 0, 200]), ZOOM_FACTOR)
    mushrooms_coords = mushrooms_coords[:, [0, 2]]
    player_coords = transform([c["coord"] for c in session.player_coords], np.array([200, 0, 200]), ZOOM_FACTOR)
    player_coords = player_coords[:, [0, 2]]
    max_coord_x = int(mushrooms_coords[:, 0].max())
    max_coord_z = int(mushrooms_coords[:, 1].max())

    with Image.new(mode="RGB", size=(max_coord_x, max_coord_z)) as im:
        draw = ImageDraw.Draw(im)
        for m in mushrooms_coords:
            draw.ellipse((tuple((m - 10).astype(int)), tuple((m + 10).astype(int))), outline="green", width=2)
        draw.line([tuple(p) for p in player_coords.astype(int)], fill=(255, 0, 0), width=5)
        for i, p in enumerate(player_coords):
            if i % (50 * 60) == 0:
                draw_cross(draw, tuple(p.astype(int)), i // (50 * 60))
        
        image_path = os.path.join(images_dir, sample.id+"_patchy.png")
        im.save(image_path)
        print(f"Image saved as {image_path}")
        
    with Image.new(mode="RGB", size=(max_coord_x, max_coord_z)) as im:
        draw = ImageDraw.Draw(im)
        draw.rectangle([0, 0, max_coord_x, max_coord_z], fill="white")
        for m in mushrooms_coords:
            draw.ellipse((tuple((m - 10).astype(int)), tuple((m + 10).astype(int))), outline="green", width=2)
        draw.line([tuple(p) for p in player_coords.astype(int)], fill=(255, 0, 0), width=5)
        
        image_path = os.path.join(images_dir, sample.id+"_patchy_notimestamp.png")
        im.save(image_path)
        print(f"Image saved as {image_path}")
        
    with Image.new(mode="RGB", size=(max_coord_x, max_coord_z)) as im:
        draw = ImageDraw.Draw(im)
        draw.line([tuple(p) for p in player_coords.astype(int)], fill=(255, 0, 0), width=5)
        
        image_path = os.path.join(images_dir, sample.id+"_patchy_nomushroom_notimestamp.png")
        im.save(image_path)
        print(f"Image saved as {image_path}")

    with Image.new(mode="RGB", size=(max_coord_x, max_coord_z)) as im:
        draw = ImageDraw.Draw(im)
        draw.line([tuple(p) for p in player_coords.astype(int)], fill=(255, 0, 0), width=5)
        for i, p in enumerate(player_coords):
            if i % (50 * 60) == 0:
                draw_cross(draw, tuple(p.astype(int)), i // (50 * 60))
        
        image_path = os.path.join(images_dir, sample.id+"_patchy_nomushroom.png")
        im.save(image_path)
        print(f"Image saved as {image_path}")

    session = sample["uniform"]
    if session is None:
        print("No session")
        continue

    ZOOM_FACTOR = 1.5

    # Transformation des coordonnées
    mushrooms_coords = transform(session.mushroom_coords, np.array([200, 0, 200]), ZOOM_FACTOR)
    mushrooms_coords = mushrooms_coords[:, [0, 2]]
    player_coords = transform([c["coord"] for c in session.player_coords], np.array([200, 0, 200]), ZOOM_FACTOR)
    player_coords = player_coords[:, [0, 2]]
    max_coord_x = int(mushrooms_coords[:, 0].max())
    max_coord_z = int(mushrooms_coords[:, 1].max())

    with Image.new(mode="RGB", size=(max_coord_x, max_coord_z)) as im:
        draw = ImageDraw.Draw(im)
        for m in mushrooms_coords:
            draw.ellipse((tuple((m - 10).astype(int)), tuple((m + 10).astype(int))), outline="green", width=2)
        draw.line([tuple(p) for p in player_coords.astype(int)], fill=(255, 0, 0), width=5)
        for i, p in enumerate(player_coords):
            if i % (50 * 60) == 0:
                draw_cross(draw, tuple(p.astype(int)), i // (50 * 60))
        
        image_path = os.path.join(images_dir, sample.id+"_uniform.png")
        im.save(image_path)
        print(f"Image saved as {image_path}")
        
    with Image.new(mode="RGB", size=(max_coord_x, max_coord_z)) as im:
        draw = ImageDraw.Draw(im)
        draw.rectangle([0, 0, max_coord_x, max_coord_z], fill="white")
        for m in mushrooms_coords:
            draw.ellipse((tuple((m - 10).astype(int)), tuple((m + 10).astype(int))), outline="green", width=2)
        draw.line([tuple(p) for p in player_coords.astype(int)], fill=(255, 0, 0), width=5)
        
        image_path = os.path.join(images_dir, sample.id+"_uniform_notimestamp.png")
        im.save(image_path)
        print(f"Image saved as {image_path}")
        
    with Image.new(mode="RGB", size=(max_coord_x, max_coord_z)) as im:
        draw = ImageDraw.Draw(im)
        draw.line([tuple(p) for p in player_coords.astype(int)], fill=(255, 0, 0), width=5)
        
        image_path = os.path.join(images_dir, sample.id+"_uniform_nomushroom_notimestamp.png")
        im.save(image_path)
        print(f"Image saved as {image_path}")

    with Image.new(mode="RGB", size=(max_coord_x, max_coord_z)) as im:
        draw = ImageDraw.Draw(im)
        draw.line([tuple(p) for p in player_coords.astype(int)], fill=(255, 0, 0), width=5)
        for i, p in enumerate(player_coords):
            if i % (50 * 60) == 0:
                draw_cross(draw, tuple(p.astype(int)), i // (50 * 60))
        
        image_path = os.path.join(images_dir, sample.id+"_uniform_nomushroom.png")
        im.save(image_path)
        print(f"Image saved as {image_path}")

    break


Image saved as c:\Users\HP\Desktop\Cours\Projet IA\images_2022\Giorgia_patchy_notimestamp.png
Image saved as c:\Users\HP\Desktop\Cours\Projet IA\images_2022\Giorgia_uniform_notimestamp.png
